# Spotify 

In [ ]:
import pandas as pd

## Completezza

In [ ]:
spotify = pd.read_csv('songs_final.csv')

In [ ]:
spotify.isnull().sum()

# Spotify stream count

## Completezza streams

In [ ]:
stream=pd.read_csv('StreamCount_final.csv')
stream.isnull().sum()

## Accuratezza

### Per alcune canzoni il nome ricevuto dall'API non corrisponde con quello preso dallo scraping

In [ ]:
streams=stream[stream['Track_Api']!=stream['Track']]
streams

### Alcune canzoni sono cambiate di posto nel tempo trascorso da quando è stata fatta la richiesta con l'API e quando è stato raccolto il numero di riproduzioni tramite scraping. Per rimediare separiamo le colonne e facciamo un merge usando ``Artist`` e ``Track`` come indice.

In [ ]:
spoti_url=streams[['Unnamed: 0','Artist','Url','Track_Api', 'Album_Api']]
spoti_stream=streams[['Artist','Track', 'Stream']]

In [ ]:
spoti_url = spoti_url.rename(columns={'Track_Api': 'Track'})
merged_df = pd.merge(spoti_url, spoti_stream, on=['Artist', 'Track'])
merged_df['Track_Api']=merged_df['Track']

In [ ]:
df = pd.merge(stream, merged_df, on=['Unnamed: 0'], how='left')

In [ ]:
df['Track_y'].fillna(df['Track_x'], inplace=True)
df['Stream_y'].fillna(df['Stream_x'], inplace=True)

In [ ]:
df.drop(['Artist_y', 'Url_y', 'Track_x', 'Stream_x', 'Track_Api_y', 'Album_Api_y'], axis=1, inplace=True)

In [ ]:
df = df.rename(columns={'Artist_x':'Artist', 'Url_x':'Url', 'Track_Api_x':'Track_Api', 'Album_Api_x':'Album' ,
                        'Stream_y':'Stream','Track_y':'Track'})

Una volta sistemate le colonne ridondanti ritroviamo il dataset corretto

In [ ]:
df

### Cercando di nuovo le canzoni dal titolo che non combacia troviamo che il nostro metodo precedente ha sistemato solo 200 canzoni. Ora troviamo che molte canzoni sono effettivamente le stesse ma con un titolo leggermente diverso oppure canzoni dal titolo totalmente diverso

In [ ]:
dif=df[df['Track_Api']!=df['Track']]
dif

### Controllo che Track_Api e Track che stanno sulla stessa riga siano le più simili tra loro in relazione alle altre canzoni dello stesso artista

Questo codice definisce due funzioni: ``string_similarity`` e ``check_similarity``.

La funzione ``string_similarity`` prende in input due stringhe a e b e calcola il rapporto tra la loro somiglianza utilizzando la funzione ``difflib.SequenceMatcher`` dalla libreria ``difflib``. La funzione rimuove prima i termini "Remastered", "remastered", "Remaster", e "remaster" dalle stringhe a e b utilizzando il metodo ``replace`` prima di effettuare il confronto.

La funzione ``check_similarity`` prende in input un dataframe ``df``, il nome della colonna dell'artista ``artist_col``, il nome della colonna di un possibile nome di traccia ``track_maybe_col`` e il nome della colonna del vero nome di traccia ``track_col``. Crea una nuova colonna chiamata "same" nel dataframe e imposta tutti i valori a ``False``. Quindi raggruppa il dataframe per il nome dell'artista e per ogni gruppo confronta ogni nome di traccia in ``track_maybe_col`` con i nomi di traccia in ``track_col``. Se la massima somiglianza di una traccia in ``track_maybe_col`` con tutte le tracce in ``track_col`` è inferiore alla somiglianza della traccia in ``track_maybe_col`` con ``track_col`` sulla stessa riga, imposta il valore della colonna "same" per quella riga a True. Infine, restituisce il dataframe modificato.

Il codice quindi chiama check_similarity su un dataframe ``dif`` e assegna il risultato a un dataframe ``simil``.

In [ ]:
import difflib
import pandas as pd

def string_similarity(a, b):
    a = a.replace('Remastered', '').replace('remastered', '').replace('Remaster', '').replace('remaster', '')
    b = b.replace('Remastered', '').replace('remastered', '').replace('Remaster', '').replace('remaster', '')
    return difflib.SequenceMatcher(None, a, b).ratio()

def check_similarity(df, artist_col, track_maybe_col, track_col):
    df['same'] = False
    for artist, group in df.groupby(artist_col):
        for i, row in group.iterrows():
            max_similarity = 0
            for j, compare_row in group.iterrows():
                if i == j:
                    continue
                similarity = string_similarity(row[track_maybe_col], compare_row[track_col])
                if similarity > max_similarity:
                    max_similarity = similarity
            if max_similarity < string_similarity(row[track_maybe_col], row[track_col]):
                df.at[i, 'same'] = True
    return df

simil = check_similarity(dif, 'Artist', 'Track_Api', 'Track')
simil

In [ ]:
simil[simil['same']==False]

In [ ]:
simil[simil['same']==True]

### Controllo le canzoni sbagliate degli artisti che ne hanno una sola sbagliata

In [ ]:
soli = simil.groupby('Artist').filter(lambda x: len(x) == 1).reset_index(drop=True)
soli

### Per questo gruppo di canzoni la nostra colonna "same", per via di come l'abbiamo definita sarà sempre vera, a prescindere da quanto siano differenti i titoli. Definiamo allora una nuova colonna che ci dica quanto sono simili i titoli presi da API e scraper

In [ ]:
import difflib
import pandas as pd

def string_similarity(row, col1, col2):
    return difflib.SequenceMatcher(None, row[col1], row[col2]).ratio()

soli['similarity'] = soli.apply(lambda row: string_similarity(row, 'Track_Api', 'Track'), axis=1)


### Decidiamo di prendere 0.5 come valore soglia 

In [ ]:
soli[soli['similarity']>0.5]

In [ ]:
soli[soli['similarity']<0.5]

### Tornando agli altri artisti con più di una canzone dissimile, in questo caso la metrica più efficace è la colonna same

In [ ]:
accompagnati = simil.groupby('Artist').filter(lambda x: len(x) > 1).reset_index(drop=True)
accompagnati['similarity'] = accompagnati.apply(lambda row: string_similarity(row, 'Track_Api', 'Track'), axis=1)
accompagnati

### Decidiamo infine di impostare un valore nullo su track e stream laddove supponiamo ci sia un errore 

Per gli artisti con una sola canzone sbagliata prendiamo per buone le tracce con similarità maggiore di 0.5

In [ ]:
mask = soli['similarity'] < 0.5
soli.loc[mask, ['Track', 'Stream']] = None
soli

Per tutti gli altri teniamo buone le tracce con valore della colonna 'same' uguale a vero

In [ ]:
mask = accompagnati['same'] == False
accompagnati.loc[mask, ['Track', 'Stream']] = None
accompagnati

### Riuniamo i dataframe e salviamo

In [ ]:
soli.drop(columns=['same','similarity'], inplace=True)
accompagnati.drop(columns=['same','similarity'], inplace=True)

In [ ]:
s_a=pd.concat([soli, accompagnati])
s_a[s_a['Stream'] == "Null"]

In [ ]:
finale = pd.merge(df, s_a, on=['Unnamed: 0'], how='left')
finale

In [ ]:
finale['Track_y'].fillna(finale['Track_x'], inplace=True)
finale['Stream_y'].fillna(finale['Stream_x'], inplace=True)

In [ ]:
finale.drop(['Unnamed: 0','Artist_y', 'Url_y', 'Track_x', 'Stream_x', 'Track_Api_y', 'Album_y'], axis=1, inplace=True)

In [ ]:
finale = finale.rename(columns={'Artist_x':'Artist', 'Url_x':'Url', 'Track_Api_x':'Track_Api', 'Album_x':'Album',
                        'Stream_y':'Stream','Track_y':'Track'})
finale

In [ ]:
finale.to_csv("StreamCount_sistemato.csv")